# Capstone Journal

**5/15/2025**:   Class meetup. Read, set, go!

**5/19/2025**:   Reviewed requirements, created project directory, base files, and drafted a vision board.

![Management Cycle Flowchart](management-cycle.png)

- NOTE: I want to give the CSV data some context by generating a new fake csv that reflects a developers finances.
- ACTION: Created `csv_faker.py` to generate 500,000 transaction records and saved them to `developer_transactions.csv`.

In [6]:
# Task 1: Loading Transactions from a CSV File

# Parse data with datetime.strptime
# Make amount negative for 'debit'
# Convert transaction_id and customer_id to integers
# Create dictionary with all fields
# Add to transactions 
# Catch FileNotFoundError and ValueError

# NOTE: These functions will all go into a `FinanceUtils` class inside the utils.py file

import csv
from datetime import datetime

def load_transactions(filename='financial_transactions.csv'):
    """Load transactions from a CSV file into a list of dictionaries."""

    transactions = []
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Convert transaction ID to integer
            row['transaction_id'] = int(row['transaction_id'])
            # Convert customer ID to integer
            row['customer_id'] = int(row['customer_id'])
            # Convert date string to datetime object
            row['date'] = datetime.strptime(row['date'], '%Y-%m-%d').date()
            # Convert amount to float
            row['amount'] = float(row['amount'])
            # Make amount negative for 'debit'
            if row['type'] == 'debit':
                row['amount'] = -row['amount']
            transactions.append(row)

    return transactions

# Test: Print the first 3 transactions
print("Loaded transactions:")
transactions = load_transactions(filename='developer_transactions.csv')
for transaction in range(3):
    print(transactions[transaction])

print(f"Total transactions loaded: {len(transactions)}")

Loaded transactions:
{'transaction_id': 1, 'date': datetime.date(2021, 11, 19), 'customer_id': 719, 'amount': 363.2, 'type': 'transfer', 'description': 'Payment to friend for dinner'}
{'transaction_id': 2, 'date': datetime.date(2024, 4, 16), 'customer_id': 674, 'amount': 456.13, 'type': 'transfer', 'description': 'Reimbursement for shared purchase'}
{'transaction_id': 3, 'date': datetime.date(2017, 4, 13), 'customer_id': 651, 'amount': 273.71, 'type': 'transfer', 'description': 'Transfer to investment account'}
Total transactions loaded: 500000


In [ ]:
# Class method:

import csv
from datetime import datetime
import logging
import os

class FinanceUtils:
    """Class to manage financial transactions with CRUD operations and analysis."""

    # Class constructor

    def load_transactions(self, filename='financial_transactions.csv'):
        """
        Load transactions from a CSV file into self.transactions.
        
        Args:
            filename (str): Path to the CSV file.
            
        Returns:
            bool: True if loading succeeds, False otherwise.
        """
        self.transactions = []
        required_columns = {'transaction_id', 'date', 'customer_id', 'amount', 'type', 'description'}

        try:
            with open(filename, mode='r', encoding='utf-8') as file:
                reader = csv.DictReader(file)

                # Check required columns
                if not required_columns.issubset(reader.fieldnames):
                    missing = required_columns - set(reader.fieldnames)
                    logging.error(f"Missing columns in CSV: {missing}")
                    print(f"Missing columns in CSV: {missing}")
                    return False
                
                for row_num, row in enumerate(reader, start=2):
                    try:
                        # Validate transaction_id
                        try:
                            transaction_id = int(row['transaction_id'])
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid transaction_id '{row['transaction_id']}'")
                            continue

                        # Validate date
                        date_str = row['date'].strip()
                        try:
                            date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid date format '{date_str}'")
                            continue

                        # Validate customer_id
                        try:
                            customer_id = int(row['customer_id'])
                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid customer_id '{row['customer_id']}'")
                            continue

                        # Validate amount
                        try:
                            amount = float(row['amount'])
                            if amount < 0:
                                logging.error(f"Row {row_num}: Negative amount '{amount}'")
                                continue

                        except ValueError:
                            logging.error(f"Row {row_num}: Invalid amount '{row['amount']}'")
                            continue

                        # Validate type
                        transaction_type = row['type'].strip().lower()
                        if transaction_type not in {'credit', 'debit', 'transfer'}:
                            logging.error(f"Row {row_num}: Invalid transaction type '{transaction_type}'")
                            continue

                        # Adjust amount for debit
                        if transaction_type == 'debit':
                            amount = -amount

                        # Validate description
                        description = row.get('description')
                        if description is None or not str(description).strip():
                            logging.error(f"Row {row_num}: Empty description")
                            continue
                        description = str(description).strip()

                        # Create transaction dictionary
                        transaction = {
                            'transaction_id': transaction_id,
                            'date': date_obj,
                            'customer_id': customer_id,
                            'amount': amount,
                            'type': transaction_type,
                            'description': description
                        }
                        self.transactions.append(transaction)

                    except KeyError as e:
                        logging.error(f"Row {row_num}: Missing column {e}")
                        continue

                print(f"Loaded {len(self.transactions)} transactions from '{filename}'.")

                # Create a backup of the original file and save it with a timestamp to /snapshots
                if not os.path.exists('snapshots'):
                    os.makedirs('snapshots')

                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                backup_filename = os.path.join('snapshots', f'backup_{timestamp}.csv')
                try:
                    with open(filename, 'rb') as src_file, open(backup_filename, 'wb') as dst_file:
                        dst_file.write(src_file.read())
                    print(f"Backup created: '{backup_filename}'")
                except Exception as e:
                    logging.error(f"Failed to create backup: {e}")

                return True
            
        except FileNotFoundError:
            logging.error(f"File '{filename}' not found.")
            print(f"File '{filename}' not found.")
            return False
        
        except csv.Error:
            logging.error(f"Malformed CSV file '{filename}'.")
            print(f"Error reading CSV file '{filename}'.")
            return False
        
        except IOError as e:
            logging.error(f"IO error reading {filename}: {e}")
            print(f"Error: IO error reading file: {e}")
            return False
        


## Testing

✅ Skips rows with invalid fields and prints a warning.  
✅ Loads and counts transactions accurately.  
✅ Checks required columns  
✅ Validates transaction_id  
✅ Validates date  
✅ Validates customer_id  
✅ Validates amount  
✅ Validates type  
✅ Validates description  

## Bonus Features

✅ Backup CSV snapshots  
✅ Handle backup errors  

